# **CA3/CA1 Neuromorphic Module – Full Visual Demo**This notebook demonstrates:### ✅ CA3→CA1 hippocampal microcircuit  ### ✅ Pattern encoding and recall  ### ✅ STDP learning  ### ✅ Multiple training patterns  ### ✅ Visualizations:- Heatmaps of neuronal firing  - Synaptic weight histograms (before & after learning)  - Pattern reconstruction from noisy cues  - CA3 vs CA1 activation comparisons  Run all cells to experience the full neuromorphic pipeline.

In [ ]:
import randomimport numpy as npimport matplotlib.pyplot as pltfrom backend.network import CA3CA1Networkfrom backend.simulation import Simulationfrom backend.plasticity import STDPfrom agents.input_agent import InputAgentfrom agents.monitoring_agent import MonitoringAgentfrom orchestrator.orchestrator import Orchestrator# Create network & connectnet = CA3CA1Network(n_ca3=100, n_ca1=100)net.connect()stdp = STDP()print("Network created.")print("CA3 neurons:", len(net.ca3))print("CA1 neurons:", len(net.ca1))print("Total synapses:", len(net.synapses))

In [ ]:
def get_activity(neurons):    return np.array([1 if n.fired else 0 for n in neurons])def get_weights(synapses):    return np.array([s.weight for s in synapses])def present_pattern(pattern, net, stdp, steps=3):    input_agent = InputAgent()    monitor = MonitoringAgent()    orch = Orchestrator(net, [input_agent, monitor])    orch.run_cycle(pattern)    sim = Simulation(net)    sim.run(steps)    # STDP update    for syn in net.synapses:        stdp.apply(syn, syn.pre.fired, syn.post.fired)    return get_activity(net.ca3), get_activity(net.ca1)

In [ ]:
num_patterns = 5n_ca3 = len(net.ca3)patterns = [[random.choice([0,1]) for _ in range(n_ca3)] for _ in range(num_patterns)]print("Generated", num_patterns, "training patterns.")

## **Initial Synaptic Weight Distribution**

In [ ]:
initial_weights = get_weights(net.synapses)plt.figure(figsize=(6,4))plt.hist(initial_weights, bins=30)plt.title("Initial Synaptic Weight Distribution")plt.xlabel("Weight")plt.ylabel("Count")plt.show()

## **Training the CA3→CA1 Network**

In [ ]:
ca3_hist = []ca1_hist = []for idx, p in enumerate(patterns):    print(f"Training with pattern {idx+1}/{num_patterns}...")    ca3, ca1 = present_pattern(p, net, stdp, steps=3)    ca3_hist.append(ca3)    ca1_hist.append(ca1)ca3_hist = np.array(ca3_hist)ca1_hist = np.array(ca1_hist)

## **Activity Heatmaps (CA3 and CA1 Across Patterns)**

In [ ]:
plt.figure(figsize=(8,4))plt.imshow(ca3_hist, aspect='auto')plt.title("CA3 Activity Across Patterns")plt.xlabel("Neuron Index")plt.ylabel("Pattern")plt.colorbar()plt.show()plt.figure(figsize=(8,4))plt.imshow(ca1_hist, aspect='auto')plt.title("CA1 Activity Across Patterns")plt.xlabel("Neuron Index")plt.ylabel("Pattern")plt.colorbar()plt.show()

## **Final Synaptic Weight Distribution After Learning**

In [ ]:
final_weights = get_weights(net.synapses)plt.figure(figsize=(6,4))plt.hist(final_weights, bins=30)plt.title("Final Synaptic Weight Distribution")plt.xlabel("Weight")plt.ylabel("Count")plt.show()print("Initial avg weight:", float(np.mean(initial_weights)))print("Final avg weight:", float(np.mean(final_weights)))

## **Pattern Recall from Noisy Input**

In [ ]:
test_id = 0original = patterns[test_id]noisy = [v if random.random() > 0.25 else 1-v for v in original]print("Original sample:", original[:25])print("Noisy sample:", noisy[:25])input_agent = InputAgent()monitor = MonitoringAgent()orch = Orchestrator(net, [input_agent, monitor])orch.run_cycle(noisy)sim = Simulation(net)sim.run(3)recalled = get_activity(net.ca1)plt.figure(figsize=(10,4))plt.subplot(1,3,1)plt.imshow(np.array(original)[None,:], aspect='auto')plt.title("Original Pattern")plt.yticks([])plt.subplot(1,3,2)plt.imshow(np.array(noisy)[None,:], aspect='auto')plt.title("Noisy Input")plt.yticks([])plt.subplot(1,3,3)plt.imshow(recalled[None,:], aspect='auto')plt.title("CA1 Recall Output")plt.yticks([])plt.tight_layout()plt.show()